In [131]:
import sqlite3
import pandas as pd

In [132]:
DATA_DIR = 'data/'

In [133]:
conn = sqlite3.connect('test.db')
cur = conn.cursor()

# Drop table from previous sessions
cur.execute('DROP TABLE IF EXISTS temperatures')
cur.execute('DROP TABLE IF EXISTS cities')

conn.commit()

In [134]:
cur.execute('''
    CREATE TABLE IF NOT EXISTS cities (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        country TEXT  
    )
''')
conn.commit()

In [135]:
cur.execute('''
                CREATE TABLE IF NOT EXISTS temperatures (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    timestamp TEXT,
                    temperature REAL,
                    city_id INTEGER
                )
            ''')
conn.commit()

In [136]:
def insert_temps_from_csv(file_path, city_name):
    # Find header row
    header_row = -1
    with open(file_path, 'r') as fp:
        for i, line in enumerate(fp):
            if line.startswith('Datum;Tid (UTC)'):
                header_row = i
                break

    if (header_row < 0):
        print("Could not find header in file!")
        return

    data = pd.read_csv(file_path, sep=";", skiprows=header_row)
    data['DatumTid'] = data['Datum'] + ' ' + data['Tid (UTC)']

    data = data[['DatumTid', 'Lufttemperatur']]

    city_id_resp = cur.execute('SELECT id FROM cities WHERE name = ?', (city_name, )).fetchone()
    if city_id_resp is None:
        print(f"City {city_name} not found, adding it!")
        resp = cur.execute('''
            INSERT INTO cities (name, country) VALUES (?, ?)
        ''', (city_name, 'Sweden'))
        city_id = resp.lastrowid
    else:
        city_id = city_id_resp[0]

    for row in data.itertuples():
        cur.execute('''
                        INSERT INTO temperatures (timestamp, temperature, city_id)
                        VALUES (?, ?, ?)
                    ''', (row[1], row[2], city_id))
    
    conn.commit()

In [137]:
insert_temps_from_csv(DATA_DIR + 'temp-uppsala.csv', 'Uppsala')
insert_temps_from_csv(DATA_DIR + 'temp-lulea.csv', 'Luleå')
insert_temps_from_csv(DATA_DIR + 'temp-goteborg.csv', 'Göteborg')

/var/folders/pg/6tvtzc815214nstb4nc45dcw0000gn/T/ipykernel_5484/1973754284.py:14: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep=";", skiprows=header_row)


City Uppsala not found, adding it!
City Luleå not found, adding it!


/var/folders/pg/6tvtzc815214nstb4nc45dcw0000gn/T/ipykernel_5484/1973754284.py:14: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep=";", skiprows=header_row)


City Göteborg not found, adding it!


In [128]:
# cur.execute('SELECT * FROM temperatures WHERE temperature > 30')
# user = cur.fetchone()

for row in cur.execute("""
                        SELECT MAX(temperatures.temperature), MIN(temperatures.temperature), cities.name FROM temperatures
                        JOIN cities ON temperatures.city_id = cities.id
                        GROUP BY cities.name
                        LIMIT 10
                       """):
    print(row)

(33.3, -23.4, 'Göteborg')
(34.0, -39.7, 'Luleå')
(33.3, -27.6, 'Uppsala')


In [115]:
print(cur.execute("SELECT 'Rast och eget arbete till 14:15!'").fetchone()[0])

Rast och eget arbete till 14:15!


In [116]:
conn.close()